<a href="https://colab.research.google.com/github/prantik-pdeb/GSoC2022-QML/blob/main/Tesk_3_(c)_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3-(c) Implementing a QCNN model in TF Quantum

The electron-photon dataset (which can be found [here](https://www.google.com/url?q=https%3A%2F%2Fgithub.com%2FML4SCI%2FML4SCI_GSoC%2Ftree%2Fmain%2FQMLHEP%2Fqcnn&sa=D&source=docs)) contains 100 samples for training and another 100 for testing, laid out as follows:

a. data["x_train"]: Training dataset of 100 32x32 images containing the particles' energy (100, 32, 32)

b. data["y_train"]:" Training labels, 0 = "photon", 1 = "electron" (100,)

c. data["x_test"]: Test dataset of 100 32x32 images containing the particles' energy (100, 32, 32)

d. data["y_test"]:" Test labels, 0 = "photon", 1 = "electron" (100,)

In [ ]:
# download tensorflow quantum
!pip install tensorflow-quantum

In [ ]:
#importing tensorflow and module dependecies 
import numpy as np
import tensorflow as tf
import importlib, pkg_resources
importlib.reload(pkg_resources)
import tensorflow_quantum as tfq
import cirq
import sympy
import operator

#import visualozation tools
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit
import matplotlib.image  as mpimg

In [ ]:
'''loading data electron-photon dataset containts 100 samples
for training and 100 sampes for test'''
'''['x_train'] and ['x_test'] training and test dataset of 100 32*32 images 
['y_train'] and ['y_test'] training and test level with 0 = photon, 1 = electron'''

dataset = np.load('/content/electron-photon.npz')

In [ ]:
with dataset as data:
  x_train = data['x_train']
  y_train = data['y_train']
  x_test = data['x_test']
  y_test= data['y_test']

# Data Visualization

In [ ]:
plt.imshow(x_train[1])
plt.colorbar()

In [ ]:
plt.imshow(x_test[1])
plt.colorbar()

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [ ]:
BATCH_SIZE = 16
SHUFFLE_BUFFER_SIZE = 18

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

# Classical Convulational Neural Network 

In [ ]:
cnn_model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64,(3,3), activation= 'relu',
                         input_shape=(32,32,1)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64,(3,3), activation= 'relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation= 'relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')                                 
])

In [ ]:
cnn_model.summary()

In [ ]:
cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
cnn_history = cnn_model.fit(train_dataset, epochs=100, validation_data = test_dataset)

In [ ]:
train_loss, train_acc = cnn_model.evaluate(train_dataset)

In [ ]:
test_loss, test_acc = cnn_model.evaluate(test_dataset)

# Result: Visualization of Classical Convolutional Neural Network

In [ ]:
train_acc = cnn_history.history['accuracy']
test_acc = cnn_history.history['val_accuracy']
train_loss = cnn_history.history['loss']
test_loss= cnn_history.history['val_loss']

epochs=range(len(train_acc)) 
plt.plot(epochs, train_acc, 'r', label= "Training Accuracy")
plt.plot(epochs, test_acc, 'b', label = "Test Accuracy")
plt.title('Training and testing accuracy')
plt.figure()

plt.plot(epochs, train_loss, 'r',label= "Training Loss")
plt.plot(epochs, test_loss, 'b', label="Test Loss")
plt.title('Training and testing loss')
plt.figure()



---
# Implementation of Quantum Convolutional Neural Network (QCNN)


In [ ]:
def crop_img(img, dimension):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, dimension))
    end = tuple(map(operator.add, start, dimension))
    slices = tuple(map(slice, start, end))
    return img[slices]

In [ ]:
crop_size = (16, 16)
x_train_cropped_size = np.array([crop_img(i, crop_size) for i in x_train])
x_test_cropped_size = np.array([crop_img(i, crop_size) for i in x_test])

In [ ]:
print('Shape of x_train dataset:',x_train_cropped_size.shape)
print('Shape of x_test dataset:',x_test_cropped_size.shape)

In [ ]:
plt.imshow(x_train_cropped_size[1])
plt.colorbar()

In [ ]:
plt.imshow(x_test_cropped_size[1])
plt.colorbar()

In [ ]:
# add the color channel into the dataset (batch_size, height, width, channel)
x_train_new_size = np.reshape(x_train_cropped_size, list(x_train_cropped_size.shape)+[1])
x_test_new_size = np.reshape(x_test_cropped_size, list(x_test_cropped_size.shape)+[1])

In [ ]:
print('Shape of x_train dataset:',x_train_new_size.shape)
print('Shape of x_test dataset:',x_test_new_size.shape)

In [ ]:
#An image size of 32x32 is much too large for current quantum computers. Resize the image down to 4x4
# Using the image resize function from tensorflow library for tf.image.resize
x_train_small = np.array([tf.image.resize(img, (16,16)).numpy() for img in x_train_new_size])
x_test_small = np.array([tf.image.resize(img, (16,16)).numpy() for img in x_test_new_size])
x_train_small = np.reshape(x_train_small, x_train_small.shape[:3])
x_test_small = np.reshape(x_test_small, x_test_small.shape[:3])
#y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
#y_test = np.asarray(y_test).astype('float32').reshape((-1,1))
y_train = y_train[:]
y_test = y_test[:]

In [ ]:
plt.imshow(x_train_small[1])
plt.colorbar()

In [ ]:
plt.imshow(x_test_small[1])
plt.colorbar()

In [ ]:
class QConv(tf.keras.layers.Layer):
    def __init__(self, filter_size, depth, activation=None, name=None, kernel_regularizer=None, **kwangs):
        super(QConv, self).__init__(name=name, **kwangs)
        self.filter_size = filter_size
        self.depth = depth
        self.learning_params = []
        self.QCNN_layer_gen()
        # self.circuit_tensor = tfq.convert_to_tensor([self.circuit])
        self.activation = tf.keras.layers.Activation(activation)
        self.kernel_regularizer = kernel_regularizer

    def _next_qubit_set(self, original_size, next_size, qubits):
        step = original_size // next_size
        qubit_list = []
        for i in range(0, original_size, step):
            for j in range(0, original_size, step):
                qubit_list.append(qubits[original_size*i + j])
        return qubit_list

    def _get_new_param(self):
        """
        return new learnable parameter
        all returned parameter saved in self.learning_params
        """
        new_param = sympy.symbols("p"+str(len(self.learning_params)))
        self.learning_params.append(new_param)
        return new_param
    
    def _QConv(self, step, target, qubits):
        """
        apply learnable gates each quantum convolutional layer level
        """
        yield cirq.CZPowGate(exponent=self._get_new_param())(qubits[target], qubits[target+step])
        yield cirq.CXPowGate(exponent=self._get_new_param())(qubits[target], qubits[target+step])
        
    def QCNN_layer_gen(self):
        """
        make quantum convolutional layer in QConv layer
        """
        pixels = self.filter_size**2
        # filter size: 2^n only for this version!
        if np.log2(pixels) % 1 != 0:
            raise NotImplementedError("filter size: 2^n only available")
        cirq_qubits = cirq.GridQubit.rect(self.filter_size, self.filter_size)
        # mapping input data to circuit
        input_circuit = cirq.Circuit()
        input_params = [sympy.symbols('a%d' %i) for i in range(pixels)]
        for i, qubit in enumerate(cirq_qubits):
            input_circuit.append(cirq.rx(np.pi*input_params[i])(qubit))
        # apply learnable gate set to QCNN circuit
        QCNN_circuit = cirq.Circuit()
        step_size = [2**i for i in range(np.log2(pixels).astype(np.int32))]
        for step in step_size:
            for target in range(0, pixels, 2*step):
                QCNN_circuit.append(self._QConv(step, target, cirq_qubits))
        # merge the circuits
        full_circuit = cirq.Circuit()
        full_circuit.append(input_circuit)
        full_circuit.append(QCNN_circuit)
        self.circuit = full_circuit # save circuit to the QCNN layer obj.
        self.params = input_params + self.learning_params
        self.op = cirq.Z(cirq_qubits[0])
        
    def build(self, input_shape):
        self.width = input_shape[1]
        self.height = input_shape[2]
        self.channel = input_shape[3]
        self.num_x = self.width - self.filter_size + 1
        self.num_y = self.height - self.filter_size + 1
        
        self.kernel = self.add_weight(name="kenel", 
                                      shape=[self.depth, 
                                             self.channel, 
                                             len(self.learning_params)],
                                     initializer=tf.keras.initializers.glorot_normal(),
                                     regularizer=self.kernel_regularizer)
        self.circuit_tensor = tfq.convert_to_tensor([self.circuit] * self.num_x * self.num_y * self.channel)
        
    def call(self, inputs):
        # input shape: [N, width, height, channel]
        # slide and collect data
        stack_set = None
        for i in range(self.num_x):
            for j in range(self.num_y):
                slice_part = tf.slice(inputs, [0, i, j, 0], [-1, self.filter_size, self.filter_size, -1])
                slice_part = tf.reshape(slice_part, shape=[-1, 1, self.filter_size, self.filter_size, self.channel])
                if stack_set == None:
                    stack_set = slice_part
                else:
                    stack_set = tf.concat([stack_set, slice_part], 1)  
        # -> shape: [N, num_x*num_y, filter_size, filter_size, channel]
        stack_set = tf.transpose(stack_set, perm=[0, 1, 4, 2, 3])
        # -> shape: [N, num_x*num_y, channel, filter_size, fiter_size]
        stack_set = tf.reshape(stack_set, shape=[-1, self.filter_size**2])
        # -> shape: [N*num_x*num_y*channel, filter_size^2]
        
        # total input citcuits: N * num_x * num_y * channel
        circuit_inputs = tf.tile([self.circuit_tensor], [tf.shape(inputs)[0], 1])
        circuit_inputs = tf.reshape(circuit_inputs, shape=[-1])
        tf.fill([tf.shape(inputs)[0]*self.num_x*self.num_y, 1], 1)
        outputs = []
        for i in range(self.depth):
            controller = tf.tile(self.kernel[i], [tf.shape(inputs)[0]*self.num_x*self.num_y, 1])
            outputs.append(self.single_depth_QCNN(stack_set, controller, circuit_inputs))
            # shape: [N, num_x, num_y] 
            
        output_tensor = tf.stack(outputs, axis=3)
        output_tensor = tf.math.acos(tf.clip_by_value(output_tensor, -1+1e-5, 1-1e-5)) / np.pi
        # output_tensor = tf.clip_by_value(tf.math.acos(output_tensor)/np.pi, -1, 1)
        return self.activation(output_tensor)
          
    def single_depth_QCNN(self, input_data, controller, circuit_inputs):
        """
        make QCNN for 1 channel only
        """
        # input shape: [N*num_x*num_y*channel, filter_size^2]
        # controller shape: [N*num_x*num_y*channel, len(learning_params)]
        input_data = tf.concat([input_data, controller], 1)
        # input_data shape: [N*num_x*num_y*channel, len(learning_params)]
        QCNN_output = tfq.layers.Expectation()(circuit_inputs, 
                                               symbol_names=self.params,
                                               symbol_values=input_data,
                                               operators=self.op)
        # QCNN_output shape: [N*num_x*num_y*channel]
        QCNN_output = tf.reshape(QCNN_output, shape=[-1, self.num_x, self.num_y, self.channel])
        return tf.math.reduce_sum(QCNN_output, 3)
        

In [ ]:
width = np.shape(x_train_small)[1]
height = np.shape(x_train_small)[2]

qcnn_model = tf.keras.models.Sequential()

#model.add(layers.Conv2D(16, (2, 2), activation='relu'))
qcnn_model.add(QConv(filter_size=2, depth=8, activation='relu', 
                     name='qconv1', input_shape=(width, height, 1)))

qcnn_model.add(tf.keras.layers.Flatten())
qcnn_model.add(tf.keras.layers.Dense(32, activation='relu'))
qcnn_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
qcnn_model.summary()

In [ ]:
SVGCircuit(QConv(filter_size=2, depth=0, activation='relu').circuit)

In [ ]:
import pydot
import graphviz
from tensorflow.keras.utils import plot_model

plot_model(qcnn_model, to_file='model_shapes.png', show_shapes=True)

In [ ]:
qcnn_model.compile(optimizer = tf.keras.optimizers.Adam(), loss = 'binary_crossentropy', 
                   metrics = ['accuracy'])

In [ ]:
qcnn_history = qcnn_model.fit(x_train_small, y_train, validation_data=(x_test_small, y_test),
                              epochs=50, steps_per_epoch=10,batch_size=5)

In [ ]:
test_loss, test_acc = qcnn_model.evaluate(x_test_small, y_test)

In [ ]:
train_loss, train_acc = qcnn_model.evaluate(x_train_small, y_train)

# Result: Visualization of Quantum Convolutional Neural Network (QNN)

In [ ]:
train_acc = qcnn_history.history['accuracy']
test_acc = qcnn_history.history['val_accuracy']
train_loss = qcnn_history.history['loss']
test_loss = qcnn_history.history['val_loss']

epochs=range(len(train_acc)) 
plt.plot(epochs, train_acc, 'r', label= "Training Accuracy")
plt.plot(epochs, test_acc, 'b', label = "Test Accuracy")
plt.title('Training and testing accuracy')
plt.figure()

plt.plot(epochs, train_loss, 'r',label= "Training Loss")
plt.plot(epochs, test_loss, 'b', label="Test Loss")
plt.title('Training and testing loss')
plt.figure()

In [ ]:
plt.plot(cnn_history.history['loss'][:25], label='Classical Convolutional Neural Network(CNN)')
plt.plot(qcnn_history.history['loss'][:25], label='Quantum Convolutional Neural Network(QCNN)')
plt.title('Quantum vs Hybrid CNN performance')
plt.xlabel('Epochs')
plt.legend()
plt.ylabel('Validation Accuracy')
plt.show()

In [ ]:
def plot_loss_curves(cnn_loss, qcnn_loss):
    fig = plt.figure()
    plt.plot(np.arange(len(cnn_loss)) + 1, cnn_loss, "rs-", label="Classical Convolutional Neural Network(CNN)")
    plt.plot(np.arange(len(qcnn_loss)) + 1, qcnn_loss, "b^-", label="Quantum Convolutional Neural Network(QCNN)")
    #plt.gca().xaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
    plt.axis([1, 50, 0, 1])
    plt.legend(fontsize=14)
    plt.xlabel("Epochs")
    plt.ylabel("Test set loss")
    plt.grid(True)
    fig.savefig('loss.png', dpi=300)

In [ ]:
plot_loss_curves(cnn_history.history['loss'], qcnn_history.history['val_loss'])

# References

1. A. Abbas, D. Sutter, C. Zoufal, A. Lucchi, A. Figalli, and S. Woerner, “The power of Quantum Neural Networks,” Nature Computational Science, vol. 1, no. 6, pp. 403–409, 2021. 
2. D. E. Rumelhart, G. E. Hinton, and R. J. Williams, “Learning representations by back-propagating errors,” Nature, vol. 323, no. 6088, pp. 533–536, 1986. 
3. “Deep Learning Specialization,” DeepLearning.AI, 24-Dec-2021. [Online]. Available: https://www.deeplearning.ai/program/deep-learning-specialization/. [Accessed: 01-Apr-2022]. 
4. “Deep learning,” Deep Learning. [Online]. Available: https://www.deeplearningbook.org/. [Accessed: 01-Apr-2022]. 
5. E. Farhi and H. Neven, “Classification with quantum neural networks on near term processors,” arXiv.org, 30-Aug-2018. [Online]. Available: https://arxiv.org/abs/1802.06002. [Accessed: 01-Apr-2022]. 
6. I. Cong, S. Choi, and M. D. Lukin, “Quantum Convolutional Neural Networks,” Nature Physics, vol. 15, no. 12, pp. 1273–1278, 2019. 
7. I. Kerenidis, J. Landman, and A. Prakash, “Quantum algorithms for deep convolutional neural networks,” arXiv.org, 04-Nov-2019. [Online]. Available: https://arxiv.org/abs/1911.01117. [Accessed: 01-Apr-2022]. 
8. M. Broughton, G. Verdon, T. McCourt, A. J. Martinez, J. H. Yoo, S. V. Isakov, P. Massey, R. Halavati, M. Y. Niu, A. Zlokapa, E. Peters, O. Lockwood, A. Skolik, S. Jerbi, V. Dunjko, M. Leib, M. Streif, D. Von Dollen, H. Chen, S. Cao, R. Wiersema, H.-Y. Huang, J. R. McClean, R. Babbush, S. Boixo, D. Bacon, A. K. Ho, H. Neven, and M. Mohseni, “TensorFlow quantum: A software framework for Quantum Machine Learning,” arXiv.org, 26-Aug-2021. [Online]. Available: https://arxiv.org/abs/2003.02989. [Accessed: 01-Apr-2022].  
9. “Quantum Convolutional Neural Network &nbsp;: &nbsp; tensorflow quantum,” TensorFlow. [Online]. Available: https://www.tensorflow.org/quantum/tutorials/qcnn. [Accessed: 01-Apr-2022]. 
10. S. Oh, J. Choi, and J. Kim, “A tutorial on quantum convolutional Neural Networks (QCNN),” arXiv.org, 20-Sep-2020. [Online]. Available: https://arxiv.org/abs/2009.09423. [Accessed: 01-Apr-2022]. 
11. “Tensorflow Quantum,” TensorFlow. [Online]. Available: https://www.tensorflow.org/quantum. [Accessed: 01-Apr-2022]. 
12. Y. LeCun and Y. Bengio, “Convolutional Networks for Images Speech and Time Series.” [Online]. Available: https://www.iro.umontreal.ca/~lisa/pointeurs/TR1312.pdf. [Accessed: 01-Apr-2022]. 
13. Y. LeCun, Y. Bengio, and G. Hinton, “Deep learning,” Nature, vol. 521, no. 7553, pp. 436–444, 2015. 